In [2]:
:dep rand = "0.6"

In [3]:
use rand;

static mut SHUT_DOWN: bool = false;

fn main() {
    loop {
        unsafe {
            SHUT_DOWN = rand::random();
        }
        print!(".");

        if unsafe {SHUT_DOWN} {
            break;
        };
    }
    println!()
}

main();

...


In [4]:
:dep libc = "0.2"

In [6]:
#![cfg(not(windows))]

use std::time::{Duration};
use std::thread::{sleep};
use libc::{SIGTERM, SIGUSR1};

static mut SHUT_DOWN: bool = false;

fn main() {
    register_signal_handlers();

    let delay = Duration::from_secs(1);

    for i in 1_usize.. {
        println!("{}", i);
        unsafe {
            if SHUT_DOWN {
                println!("*");
                return;
            }
        }
        sleep(delay);

        let signal = if i > 2 {
            SIGTERM
        } else {
            SIGUSR1
        };

        unsafe {
            libc::raise(signal);
        }
    }
    unreachable!();
}

fn register_signal_handlers() {
    unsafe {
        libc::signal(SIGTERM, handle_sigterm as usize);
        libc::signal(SIGUSR1, handle_sigusr1 as usize);
    }
}

#[allow(dead_code)]
fn handle_sigterm(_signal: i32) {
    register_signal_handlers();
    println!("SIGTERM");

    unsafe {
        SHUT_DOWN = true;
    }
}

#[allow(dead_code)]
fn handle_sigusr1(_signal: i32) {
    register_signal_handlers();
    println!("SIGUSR1");
}

main();

1
SIGUSR1
2
SIGUSR1
3
SIGTERM
4
*


In [7]:
fn noop() {}

fn main() {
    let fn_ptr = noop as usize;

    println!("noop as usize: 0x{:x}", fn_ptr);
}

main();

noop as usize: 0x7f11ca286790


In [8]:
fn noop() {}

fn main() {
    let fn_ptr = noop as usize;
    let typed_fn_ptr = noop as *const fn() -> ();

    println!("noop as usize: 0x{:x}", fn_ptr);
    println!("noop as *const T: {:p}", typed_fn_ptr);
}

main();

noop as usize: 0x7f11ca282880
noop as *const T: 0x7f11ca282880


In [9]:
use libc::{signal, raise};
use libc::{SIG_DFL, SIG_IGN, SIGTERM}

fn main() {
    unsafe {
        signal(SIGTERM, SIG_IGN);
        raise(SIGTERM);
    }

    println!("ok");

    unsafe {
        signal(SIGTERM, SIG_DFL);
        raise(SIGTERM);
    }

    println!("not ok");
}

main();

ok


Error: Child process terminated with status: signal: 15

In [10]:
fn print_depth(depth: usize) {
    for _ in 0..depth {
        print!("#");
    }
    println!("");
}

fn dive(depth: usize, max_depth: usize) {
    print_depth(depth);
    if depth >= max_depth {
        return;
    } else {
        dive(depth+1, max_depth);
    }
    print_depth(depth);
}

dive(0,5);


#
##
###
####
#####
####
###
##
#

